In [59]:
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [60]:
t = 120
t1 = 40
n1 = 5
n2 = 25
n3 = 50
rf_rate=0
lowermu=.05/12
uppermu=.15/12
lowersigma = .05/sqrt(12)
uppersigma = .6/sqrt(12)
targetcorrelation = .5
targetsd=.1/sqrt(12)


In [72]:
def generate_true_params(N,lmu=lowermu, umu=uppermu, lsig=lowersigma, usig=uppersigma, tcor=targetcorrelation):
    mu = np.random.uniform(lmu, umu, N)
    sigma = np.random.uniform(lsig, usig, N)

    cov_matrix = np.ones((N, N)) * targetcorrelation
    np.fill_diagonal(cov_matrix, 1)
    diagsig = np.diag(sigma)
    Covariancemat = diagsig @ cov_matrix @ diagsig
    return mu, Covariancemat


In [62]:
def generate_returns(mu, cov_matrix, T,N):
    returns = np.zeros((T,N))
    for t in range(T):
        returns[t]=np.random.multivariate_normal(mu,cov_matrix)
    return returns

In [71]:
def calculate_realized_portfolio_return(returns, w, target_std):
    portfolio_var = (np.dot(np.dot(w, returns.T), w))
    leverage = target_std**2 / portfolio_var
    return np.dot(w, returns) * np.sqrt(leverage)
#* leverage

def estimate_parameters_and_portfolio_returns(returns, T1,target_std, T):
    realized_returns = []
    tangency_portfolio_weights = []
    for t in range(T1, T):
        # Estimate expected returns and covariance matrix
        returns_up_to_t = returns[:t]
        estimated_mu = np.mean(returns_up_to_t, axis=0)
        estimated_cov_matrix = np.cov(returns_up_to_t.T)
        
        # Calculate tangency portfolio weights
        inv_cov_matrix = np.linalg.inv(estimated_cov_matrix)
        excess_returns = estimated_mu - rf_rate
        w = np.dot(inv_cov_matrix, excess_returns) / np.sum(np.dot(inv_cov_matrix, excess_returns))
        
        # Store tangency portfolio weights
        tangency_portfolio_weights.append(w)
        
        # Calculate realized portfolio return with target standard deviation
        realized_return = calculate_realized_portfolio_return(returns[t], w, target_std)
        realized_returns.append(realized_return)
    return realized_returns

In [70]:
mumatrix, covmatrix = generate_true_params(5)
x = generate_returns(mumatrix,covmatrix,120,5)

print(estimate_parameters_and_portfolio_returns(x, 40,targetsd,120))




[array([        nan, -0.00774753,         nan,         nan, -0.02128933]), array([       nan,        nan, 0.00261665, 0.01518499,        nan]), array([        nan, -0.01653192,         nan,         nan, -0.0306094 ]), array([        nan, -0.01741917,         nan,         nan, -0.0289228 ]), array([-0.04673676,         nan,         nan, -0.12386036, -0.03552088]), array([       nan,        nan, 0.00691156, 0.03077362,        nan]), array([       nan,        nan, 0.00416836, 0.01861565,        nan]), array([-0.01179417, -0.00945566,         nan,         nan, -0.00900724]), array([-0.04827309, -0.03387083,         nan,         nan, -0.03540803]), array([-0.02545311, -0.02164223,         nan,         nan, -0.05764159]), array([-0.03296712, -0.03734605,         nan,         nan, -0.04223878]), array([        nan, -0.03088355,         nan,         nan, -0.06798323]), array([0.03559848,        nan, 0.00450272, 0.04579283,        nan]), array([        nan, -0.0311945 ,         nan,         nan

C:\Users\Owner\AppData\Local\Temp\ipykernel_49592\2523104012.py:4: RuntimeWarning: invalid value encountered in sqrt
  return np.dot(w, returns) * np.sqrt(leverage)
